## Generate Metacognitive Labels using the Della-Inference API

In [1]:
import requests
import json
import pandas as pd
import os

In [8]:
# port = 12257
port = 48002
url = f"http://localhost:{port}/v1/chat/completions"

headers={
    "Content-Type": "application/json",
    "Authorization": "token-abc123"
}

model_name = 'meta-llama/Meta-Llama-3.1-70B-Instruct'

In [9]:
def get_skill_labels_prompt(opening_statement, question):
    #     system_prompt = """You are a legal expert tasked with analyzing a Supreme Court oral argument question in the context of a provided opening statement. Your goal is to label the question with the minimum number of descriptive characteristics (tags) necessary to fully capture its purpose or nature. Each tag should be concise, written in lower case, and formatted like a Python dictionary key (e.g., questioning_statutory_interpretation).

    #         ### Instructions:
    #         1. Identify Tags: Assign as many tags as necessary but only the minimum required to describe the essence of the question.
    #         2. Provide Reasons: For each tag, explain why it applies to the question.
    #         3. Format Output: Present your response in a structured JSON format.

    #         ### Output format:
    #         Your response must follow this JSON format:
    #         {
    #         "tags": [
    #             {
    #             "name": "<name_of_characteristic>",
    #             "reason": "<reason_for_the_characteristic>"
    #             },
    #             ...
    #         ]
    #         }

    #         ### Example:
    #         Opening Statement: "<speaker>John Doe</speaker> <text>Mr. Chief Justice, and may it please the Court: The key question in this case is whether an employer's internal policy can override an employee's federally protected rights under the Whistleblower Protection Act. Our position is that the Act’s protections are paramount, ensuring employees can report wrongdoing without fear of retaliation, regardless of internal company rules. We ask the Court to affirm this vital safeguard.</text>"
    #         Question: "Wouldn’t your interpretation effectively prevent companies from enforcing any internal policies related to confidentiality?"

    #         ### Response:
    #         {
    #         "tags": [
    #             {
    #             "name": "testing_limits",
    #             "reason": "The question explores the boundaries of the advocate’s argument by asking about its impact on internal company policies."
    #             },
    #             {
    #             "name": "evaluating_policy_consequences",
    #             "reason": "It examines the broader implications of the interpretation for company operations and confidentiality rules."
    #             }
    #         ]
    #         }

    #     """
    system_prompt = """You are a legal expert. Consider the following Supreme Court oral argument question in context of the provided opening statement. Label this question with a property that best describes the purpose or nature of the question. The property name should be lower case letters only. The property name should be very descriptive and you may use multiple words to describe the purpose of the question. If you do use multiple words per question, then join them by an underscore. Basically, you should be able to use the property name as a dictionary key in python.
        Your property label should capture the essence of what the question is doing in context of the provided opening statement. Provide a reason for the property you have chosen to explain why it applies to this question.

        ### Output format:
        Your response must follow this JSON format:
        {
            "property": "<name_of_property>",
            "reasoning": "<brief_explanation_for_the_choice_of_property>"
        }


        ### Example:
        Opening Statement: "<speaker>John Doe</speaker> <text>Mr. Chief Justice, and may it please the Court: The key question in this case is whether an employer's internal policy can override an employee's federally protected rights under the Whistleblower Protection Act. Our position is that the Act’s protections are paramount, ensuring employees can report wrongdoing without fear of retaliation, regardless of internal company rules. We ask the Court to affirm this vital safeguard.</text>"
        Question: "Wouldn’t your interpretation effectively prevent companies from enforcing any internal policies related to confidentiality?"

        ### Response:

        {
            "property": "evaluating_policy_consequences",
            "reasoning": "The questions examines the broader implications of the interpretation for company operations and confidentiality rules."
        }
    """
    


    user_prompt = f"""### Your Task:
        Opening Statement: {opening_statement}
        Question: {question}

        ### Response:
    """

    messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": user_prompt}
        ]
    return messages

In [10]:
def get_model_response(messages):

    payload = {
        "model": model_name,
        "messages": messages
    }

    response = requests.post(url, data=json.dumps(payload), headers=headers)
    return response

def parse_response(response):
    decoded = response.content.decode('utf-8')
    response_data = json.loads(decoded)
    content = response_data['choices'][0]['message']['content']

    # try:
    #     tags = json.dumps(content)
    # except Exception as e:
    #     print(f"Unable to jsonify response, saving string itself. ERROR: {e}")
    #     tags = str(content)
    # return tags
    return content

In [11]:
def generate_skill_labels(opening_statement, question):
    messages = get_skill_labels_prompt(opening_statement, question)
    response = get_model_response(messages)
    tags = parse_response(response)
    return tags

### Get labels for historical questions

In [21]:
### CHANGE THIS TO GENERATE FOR DIFF CHUNKS

### NEXT TODO IS 2018, chunk 2
year = '2023'
chunk = '0'


# # input_dir = f'../datasets/historical_questions/{year}/slices'
# input_filename = f'{year}_{chunk}_questions_from_current_justices'
# input_fp = f'{input_dir}/{input_filename}.csv'


input_filename = f'{year}_{chunk}_questions_from_current_justices'
model = model_name.split('/')[-1]
input_dir = f'../datasets/coherence/'
input_subdir = os.path.join(input_dir, model)
os.makedirs(input_subdir, exist_ok=True)
input_fp = f'{input_subdir}/{input_filename}.csv'



out_dir = f'../datasets/llm_outputs/metacog/'
model = model_name.split('/')[-1]
out_subdir = os.path.join(out_dir, model)
os.makedirs(out_subdir, exist_ok=True)
out_fp = f'{out_subdir}/{input_filename}_coherent_metacog_labels_single.csv'

In [22]:
df = pd.read_csv(input_fp)
df.head()

,Unnamed: 0,transcript_id,question_addressee,justice,question_text,opening_statement,year,label
0,226,2023.22-807-t01,petitioner,Clarence Thomas,"Mr. Gore, we review this for clear error. And ...",<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,coherent
1,227,2023.22-807-t01,petitioner,Elena Kagan,Did your expert present an alternative study w...,<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,incoherent
2,228,2023.22-807-t01,petitioner,Sonia Sotomayor,I think you end up in a very poor starting poi...,<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,incoherent
3,229,2023.22-807-t01,petitioner,"Samuel A. Alito, Jr.","Well, one may read Cooper a different way. Coo...",<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,Coherent
4,230,2023.22-807-t01,petitioner,Ketanji Brown Jackson,"Well, let me ask you, how could there be direc...",<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,coherent


#### Run on slice and save

In [22]:
df_coherent = df[df['label'] == 'coherent']
df_coherent = df_coherent.copy()
df_coherent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537 entries, 0 to 995
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          537 non-null    int64 
 1   transcript_id       537 non-null    object
 2   question_addressee  537 non-null    object
 3   justice             537 non-null    object
 4   question_text       537 non-null    object
 5   opening_statement   537 non-null    object
 6   year                537 non-null    int64 
 7   label               537 non-null    object
dtypes: int64(2), object(6)
memory usage: 37.8+ KB


In [23]:
df_coherent['skill_labels_raw'] = df_coherent.apply(
    lambda row: generate_skill_labels(row['opening_statement'], row['question_text']), axis=1
)
df_coherent.head()

,Unnamed: 0,transcript_id,question_addressee,justice,question_text,opening_statement,year,label,skill_labels_raw
0,226,2023.22-807-t01,petitioner,Clarence Thomas,"Mr. Gore, we review this for clear error. And ...",<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,coherent,"{\n ""property"": ""assessing_evidentiary_weig..."
4,230,2023.22-807-t01,petitioner,Ketanji Brown Jackson,"Well, let me ask you, how could there be direc...",<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,coherent,"{\n ""property"": ""examining_evidence_require..."
5,231,2023.22-807-t01,petitioner,Ketanji Brown Jackson,But are you requiring that? Could we ever -- c...,<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,coherent,"{\n ""property"": ""evaluating_evidentiary_req..."
6,232,2023.22-807-t01,petitioner,Sonia Sotomayor,"Putting that aside, there were alternative map...",<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,coherent,"{\n ""property"": ""identifying_additional_leg..."
8,234,2023.22-807-t01,petitioner,Elena Kagan,"That's the legal error, is that they didn't co...",<speaker>John M. Gore</speaker><text>Mr. Chief...,2023,coherent,"{\n ""property"": ""identifying_error_in_legal..."


In [24]:
df_coherent.to_csv(out_fp, index=False)

#### Loop historical data

In [12]:
year = 2021
for i in range(1, 4):
    chunk = i
    input_filename = f'{year}_{chunk}_questions_from_current_justices'
    model = model_name.split('/')[-1]
    input_dir = f'../datasets/coherence/'
    input_subdir = os.path.join(input_dir, model)
    os.makedirs(input_subdir, exist_ok=True)
    input_fp = f'{input_subdir}/{input_filename}.csv'

    out_dir = f'../datasets/llm_outputs/metacog/'
    model = model_name.split('/')[-1]
    out_subdir = os.path.join(out_dir, model)
    os.makedirs(out_subdir, exist_ok=True)
    out_fp = f'{out_subdir}/{input_filename}_coherent_metacog_labels_single.csv'
    
    df = pd.read_csv(input_fp)
    df_coherent = df[df['label'] == 'coherent']
    df_coherent = df_coherent.copy()
    
    print(f"Read file: {input_fp}")
    print(f"Running on chunk {i}")
    df_coherent['skill_labels_raw'] = df_coherent.apply(
        lambda row: generate_skill_labels(row['opening_statement'], row['question_text']), axis=1
    )
    df_coherent.to_csv(out_fp, index=False)
    print(f"Saved results for chunk {i} to {out_fp}")

Read file: ../datasets/coherence/Meta-Llama-3.1-70B-Instruct/2021_1_questions_from_current_justices.csv
Running on chunk 1
Saved results for chunk 1 to ../datasets/llm_outputs/metacog/Meta-Llama-3.1-70B-Instruct/2021_1_questions_from_current_justices_coherent_metacog_labels_single.csv
Read file: ../datasets/coherence/Meta-Llama-3.1-70B-Instruct/2021_2_questions_from_current_justices.csv
Running on chunk 2


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
year = 2022
for i in range(3, 5):
    chunk = i
    input_filename = f'{year}_{chunk}_questions_from_current_justices'
    model = model_name.split('/')[-1]
    input_dir = f'../datasets/coherence/'
    input_subdir = os.path.join(input_dir, model)
    os.makedirs(input_subdir, exist_ok=True)
    input_fp = f'{input_subdir}/{input_filename}.csv'

    out_dir = f'../datasets/llm_outputs/metacog/'
    model = model_name.split('/')[-1]
    out_subdir = os.path.join(out_dir, model)
    os.makedirs(out_subdir, exist_ok=True)
    out_fp = f'{out_subdir}/{input_filename}_coherent_metacog_labels_single.csv'
    
    df = pd.read_csv(input_fp)
    df_coherent = df[df['label'] == 'coherent']
    df_coherent = df_coherent.copy()
    
    print(f"Running on chunk {i}")
    df_coherent['skill_labels_raw'] = df_coherent.apply(
        lambda row: generate_skill_labels(row['opening_statement'], row['question_text']), axis=1
    )
    df_coherent.to_csv(out_fp, index=False)
    print(f"Saved results for chunk {i} to {out_fp}")

Running on chunk 3
Saved results for chunk 3 to ../datasets/llm_outputs/metacog/Meta-Llama-3.1-70B-Instruct/2022_3_questions_from_current_justices_coherent_metacog_labels_single.csv
Running on chunk 4


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [6]:
for i in range(1, 4):
    print(i)

1
2
3


#### Sanity test with subsample

In [ ]:
df_new = df.head(2).copy()
df_new['label'] = df_new.apply(
    lambda row: get_coherence_judgments(row['opening_statement'], row['question_text']), axis=1
)
df_new

In [ ]:
df_new.to_csv(out_fp, index=False)

In [6]:
for i in range(4, 5):
    print(i)

4


### Get labels for 2024

#### Get labels for 2024 'coherent' questions

In [15]:
input_fp = '../datasets/2024_all_questions_coherence_labeled_Meta-Llama-3.1-70B-Instruct.csv'
df = pd.read_csv(input_fp)
df.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label
0,2024.23-621-t01,petitioner,Clarence Thomas,You --can a consent decree or a default judgm...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
1,2024.23-621-t01,petitioner,Clarence Thomas,But I thought your argument hinged on a court...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent
2,2024.23-621-t01,petitioner,"John G. Roberts, Jr.",What do you do with the formulation by your f...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent
3,2024.23-621-t01,petitioner,Elena Kagan,"Well, it's -- it's true that it's only a lik...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent
4,2024.23-621-t01,petitioner,Ketanji Brown Jackson,But it's not that determination that's making...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent


In [16]:
df_coherent = df[df['label'] == 'coherent']
df_coherent = df_coherent.copy()

In [17]:
# GENERATE FOR ALL
df_coherent['skill_labels_raw'] = df_coherent.apply(
    lambda row: generate_skill_labels(row['opening_statement'], row['question_text']), axis=1
)
df_coherent.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label,skill_labels_raw
1,2024.23-621-t01,petitioner,Clarence Thomas,But I thought your argument hinged on a court...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""tags"": [\n {\n ""name"": ""clarifyi..."
3,2024.23-621-t01,petitioner,Elena Kagan,"Well, it's -- it's true that it's only a lik...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""tags"": [\n {\n ""name"": ""weighing..."
4,2024.23-621-t01,petitioner,Ketanji Brown Jackson,But it's not that determination that's making...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""tags"": [\n {\n ""name..."
5,2024.23-621-t01,petitioner,Ketanji Brown Jackson,When you think about the difference between m...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""tags"": [\n {\n ""name..."
7,2024.23-621-t01,petitioner,Ketanji Brown Jackson,But didn't Sole open -- leave open that --th...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""tags"": [\n {\n ""name..."


Save File:

In [18]:
out_fp = '../datasets/2024_coherent_metacog_labels_multi.csv'
df_coherent.to_csv(out_fp, index=False)

#### Get labels for 2024 'incoherent' questions

In [19]:
input_fp = '../datasets/2024_all_questions_coherence_labeled_Meta-Llama-3.1-70B-Instruct.csv'
df = pd.read_csv(input_fp)
df_incoherent = df[df['label'] == 'incoherent']
df_incoherent = df_incoherent.copy()

In [20]:
# GENERATE FOR ALL
df_incoherent['skill_labels_raw'] = df_incoherent.apply(
    lambda row: generate_skill_labels(row['opening_statement'], row['question_text']), axis=1
)
df_incoherent.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label,skill_labels_raw
0,2024.23-621-t01,petitioner,Clarence Thomas,You --can a consent decree or a default judgm...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n ""tags"": [\n {\n ""name..."
2,2024.23-621-t01,petitioner,"John G. Roberts, Jr.",What do you do with the formulation by your f...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n ""tags"": [\n {\n ""name"": ""evaluati..."
6,2024.23-621-t01,petitioner,Ketanji Brown Jackson,What about the Chief Justice's example? In t...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n""tags"": [\n {\n ""name"": ""referencing_..."
8,2024.23-621-t01,petitioner,Elena Kagan,"And -- and -- and couldn't a state do that, y...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n ""tags"": [\n {\n ""name..."
11,2024.23-621-t01,petitioner,"Samuel A. Alito, Jr.",If there is very strong evidence that the gov...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",incoherent,"{\n ""tags"": [\n {\n ""name..."


Save file:

In [21]:
out_fp = '../datasets/2024_incoherent_metacog_labels_multi.csv'
df_coherent.to_csv(out_fp, index=False)

### Test on sample

In [10]:
# # test on subsample
df_new = df_coherent.head(2).copy()
df_new['skill_labels_raw'] = df_new.apply(
    lambda row: generate_skill_labels(row['opening_statement'], row['question_text']), axis=1
)
df_new

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text,label,skill_labels_raw
1,2024.23-621-t01,petitioner,Clarence Thomas,But I thought your argument hinged on a court...,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""tags"": [\n {\n ""name..."
3,2024.23-621-t01,petitioner,Elena Kagan,"Well, it's -- it's true that it's only a lik...",<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",coherent,"{\n ""tags"": [\n {\n ""name..."


In [14]:
sample_resp = json.loads(df_new['skill_labels_raw'][3])
sample_resp

{'tags': [{'name': 'challenging_interpretation',
   'reason': "The question disputes the advocate's interpretation of a preliminary injunction's impact on prevailing party status."},
  {'name': 'arguing_by_analogy',
   'reason': "It draws an analogy to a consent decree case, suggesting that the legislature's action is similar to a unilateral settlement."},
  {'name': 'evaluating_outcome_evidence',
   'reason': 'It emphasizes the fact that the parties obtained everything they wanted in the interim period, implying that this outcome should influence the determination of prevailing party status.'},
  {'name': 'questioning_policy_implications',
   'reason': "It raises questions about the practical implications of the advocate's proposed bright-line rule, specifically with regards to fee allocation."},
  {'name': 'weighing_circumstantial_evidence',
   'reason': 'It considers various factors (likelihood of success, interim relief, and legislative action) to argue that they collectively suppo